In [ ]:
import os

os.chdir('/Users/willychien/20FDS/Assignment/')
print(os.getcwd())

### Read in impure dataset

In [ ]:
import pandas as pd
import re
import numpy as np
import string 
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer
import html.parser as html
import nltk
from nltk.corpus import stopwords 
pd.options.display.max_colwidth = None
pd.options.display.max_columns = None

filename = 'geo_tweets.csv'
df = pd.read_csv(filename, encoding ='utf_8')
print("nr duplicated: ", df.duplicated().sum())
print("### shape ###")
print("nr rows:", df.shape[0], "| nr cols:", df.shape[1])

nr duplicated:  13
### shape ###
nr rows: 30008 | nr cols: 10


### Identifiying Tags we want to keep

In [ ]:
hillary_positive_tags = ['#imwithher', '#clintonkaine16', '#voteclinton', '#hillafornia', '#hilyes', '#voteblue']
hillary_negative_tags = ['#neverhillary', '#crookedhillary', '#hillaryshealth', '#hilliary', '#hillary4prison']
hillary_tags = ['@hillaryclinton', '#hillaryclinton', '#hillary', '#clinton', 'clinton', 'hillary'] + hillary_positive_tags + hillary_negative_tags

trump_positive_tags = ['#trumppence16', '#maga', '#trumptrain', '#makeamericagreatagain']
trump_negative_tags = ['#dumptrump', '#nevertrump', '#conmanbaby', '#fuckyoudonald']
trump_tags = ['@realdonaldtrump', '#trump', '#donaldtrump', 'trump', 'donald'] + trump_positive_tags + trump_negative_tags

tag_list = hillary_tags + trump_tags

def has_tag(text: str) -> bool:
    text_in_lower_case = text.lower()
    return any(word in text_in_lower_case for word in tag_list)

df['text_has_tag'] = df['text'].apply(has_tag)
print("### shape ###")
print("nr rows:", df.shape[0], "| nr cols:", df.shape[1])

### shape ###
nr rows: 30008 | nr cols: 11


In [ ]:
# Removing the rows where 'text_has_tag' column is false
tag_false = df[df['text_has_tag'] == False].index
df.drop(tag_false , inplace=True)
print("### shape ###")
print("nr rows:", df.shape[0], "| nr cols:", df.shape[1])
print("### describe ###")
print(df.describe(include='all'))

### shape ###
nr rows: 27621 | nr cols: 11
### describe ###
                    text in_reply_to_screen_name  followers_count  \
count              27621                   18138     2.761600e+04   
unique             27392                    4499              NaN   
top     @realDonaldTrump         realDonaldTrump              NaN   
freq                  29                    5845              NaN   
mean                 NaN                     NaN     2.923926e+03   
std                  NaN                     NaN     6.673818e+04   
min                  NaN                     NaN     0.000000e+00   
25%                  NaN                     NaN     8.000000e+01   
50%                  NaN                     NaN     3.860000e+02   
75%                  NaN                     NaN     1.590000e+03   
max                  NaN                     NaN     1.085954e+07   

        retweet_count                      created_at        id_str  \
count         27616.0                   

### Drop rows where country != United States

In [ ]:
# Get names of indexes for which column Age has value 30
indexNames = df[ df['country'] != "United States" ].index
# Drop rows that do not contain the US
df.drop(indexNames , inplace=True)
print("### shape ###")
print("nr rows:", df.shape[0], "| nr cols:", df.shape[1])
print("### describe ###")
print(df.describe(include='all'))

### shape ###
nr rows: 24637 | nr cols: 11
### describe ###
                    text in_reply_to_screen_name  followers_count  \
count              24637                   16210     2.463700e+04   
unique             24430                    3947              NaN   
top     @realDonaldTrump         realDonaldTrump              NaN   
freq                  26                    5387              NaN   
mean                 NaN                     NaN     2.776382e+03   
std                  NaN                     NaN     6.992751e+04   
min                  NaN                     NaN     0.000000e+00   
25%                  NaN                     NaN     7.500000e+01   
50%                  NaN                     NaN     3.620000e+02   
75%                  NaN                     NaN     1.584000e+03   
max                  NaN                     NaN     1.085954e+07   

        retweet_count                      created_at        id_str  \
count         24637.0                   

### Investigate new US filtered df

In [ ]:
#check for any NAs in the whole dataframe
print(df.isnull().values.any())

# SEEING DIFFERENT TYPES OF PLACE NAMES WITH DIFFERENT FORMATS 
# WILL GET RID OF EVERYTHING THAT IS NOT IN 'CITY' FORMAT

print(df['place_type'].unique())
# ['city' 'admin' 'neighborhood' 'poi' 'country']

df[df['place_type'] == 'city'] #436,470
df[df['place_type'] == 'admin'] #150547  # State, Country
df[df['place_type'] == 'neighborhood'] #1212 # Neighbourhood + City or Neighbourhood + State
df[df['place_type'] == 'poi'] #2129 #Random
df[df['place_type'] == 'country'] #1632 # Country Level - US

# Get names of indexes for which column Age has value 30
indexNames = df[ df['place_type'] != "city" ].index
# Drop rows that do not contain 'city' as 'place type'
df.drop(indexNames , inplace=True)

print("### shape ###")
print("nr rows:", df.shape[0], "| nr cols:", df.shape[1])
print("### describe ###")
print(df.describe(include='all'))

True
['city' 'admin' 'neighborhood' 'poi' 'country']
### shape ###
nr rows: 18247 | nr cols: 11
### describe ###
                    text in_reply_to_screen_name  followers_count  \
count              18247                   12143     1.824700e+04   
unique             18100                    3108              NaN   
top     @realDonaldTrump         realDonaldTrump              NaN   
freq                  20                    4017              NaN   
mean                 NaN                     NaN     2.735736e+03   
std                  NaN                     NaN     8.100674e+04   
min                  NaN                     NaN     0.000000e+00   
25%                  NaN                     NaN     7.300000e+01   
50%                  NaN                     NaN     3.470000e+02   
75%                  NaN                     NaN     1.402000e+03   
max                  NaN                     NaN     1.085954e+07   

        retweet_count                      created_at     

In [ ]:
# Split full_place_name into City and State

df[['city','state']] = df.full_place_name.str.split(",",expand=True,)

print("### shape ###")
print("nr rows:", df.shape[0], "| nr cols:", df.shape[1])
print("### describe ###")
print(df.describe(include='all'))
print("### df.isnull().sum() ###")
print(df.isnull().sum())

### shape ###
nr rows: 18247 | nr cols: 13
### describe ###
                    text in_reply_to_screen_name  followers_count  \
count              18247                   12143     1.824700e+04   
unique             18100                    3108              NaN   
top     @realDonaldTrump         realDonaldTrump              NaN   
freq                  20                    4017              NaN   
mean                 NaN                     NaN     2.735736e+03   
std                  NaN                     NaN     8.100674e+04   
min                  NaN                     NaN     0.000000e+00   
25%                  NaN                     NaN     7.300000e+01   
50%                  NaN                     NaN     3.470000e+02   
75%                  NaN                     NaN     1.402000e+03   
max                  NaN                     NaN     1.085954e+07   

        retweet_count                      created_at        id_str  \
count         18247.0                   

In [ ]:
# Why would I have duplicate rows in the dataframe?!

# Select duplicate rows except first occurrence based on all columns
print("nr duplicated before: ", df.duplicated().sum())
df.drop_duplicates(inplace = True)
print("nr duplicated after: ", df.duplicated().sum())
print("### shape ###")
print("nr rows:", df.shape[0], "| nr cols:", df.shape[1])

nr duplicated before:  7
nr duplicated after:  0
### shape ###
nr rows: 18240 | nr cols: 13


## Begin Tweet Cleaning
### 1. Remove hyperlinks

In [ ]:
def extract_link(text):
    """ A function that extracts the hyperlinks tweet content """
    regex = r'https?://[^\s<>"]+|www\.[^\s<>"]+'
    match = re.search(regex, text)
    if match:
        return match.group()
    return ''

In [ ]:
def remove_link(text):
    result = re.sub(r"http\S+", "", text)
    return result

df['text_no_http'] = df['text'].apply(remove_link)

In [ ]:
# After removing hyperlinks - are there blank lines? NaN?
print("null text: ", df['text_no_http'].isnull().sum())
print("empty text: ", df['text_no_http'].eq('').sum())
print("nr rows:", df.shape[0], "| nr cols:", df.shape[1])

null text:  0
empty text:  0
nr rows: 18240 | nr cols: 14


### 2. Tokenise the dataset

In [ ]:
def tokenize(text):
    #tokenizer = RegexpTokenizer(r'\w+|[!,\-,]')
    tokenizer = TweetTokenizer()
    cleaned_tweet_tokens = tokenizer.tokenize(text.lower())
    return cleaned_tweet_tokens

from nltk.tokenize import TweetTokenizer 
 
df['tokens'] = df['text_no_http'].apply(tokenize)

In [ ]:
# Check tokenized tweets

for row in df.tokens[0:10]:  # iterating through the rows of the object column
    print(row, '\n')


['@theblaze', '@realdonaldtrump'] 

['@barackobama', '@fbi', '@lorettalynch', 'all', 'in', 'collusion', 'together', '#nojustice', '@realdonaldtrump', '#trumppence'] 

['@theblaze', '@realdonaldtrump'] 

['#cnn', '#newday', 'clear', '#trump', 'deliberately', 'throwing', 'this', 'race', ',', 'in', '2007', 'he', 'knew', 'that', '#isis', 'and', 'destabilization', 'of', 'mideast', 'started', 'w', '/', 'iraq', 'invasion'] 

['@realdonaldtrump', ',', 'you', "wouldn't", 'recognize', 'a', 'lie', 'if', 'it', 'came', 'from', 'your', 'own', 'mouth', ',', 'and', 'they', 'do', 'continually', '.', '#nevertrump'] 

['"', 'kid', ',', 'you', 'know', ',', 'suing', 'someone', '?', 'thats', 'the', 'most', 'beautiful', 'thing', '1', 'human', 'being', 'could', 'do', 'to', 'another', 'human', 'being', '"', '@funnyordie', '@realdonaldtrump', '😂', '💩', 's'] 

['@hillaryclinton', 'you', 'are', 'the', 'co-founder', 'of', 'isis', ',', 'you', 'crooked', ',', 'evil', ',', 'lying', ',', 'witch', '.', 'how', 'can', 'y

### 3. Remove Stopwords & Special Characters

In [ ]:
import nltk
from nltk.corpus import stopwords

stopwords_vocabulary = stopwords.words('english')
necessary_stop_words = ['he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself']
unecessary_stop_words = [word for word in stopwords_vocabulary if word not in necessary_stop_words]
print('stopwords_vocabulary contains she', 'she' in stopwords_vocabulary)
print('unecessary_stop_words contains she', 'she' in unecessary_stop_words)
df['no_stopwords'] = df['tokens'].apply(lambda x: [i for i in x if i.lower() not in unecessary_stop_words])
print("### shape ###")
print("nr rows:", df.shape[0], "| nr cols:", df.shape[1])

stopwords_vocabulary contains she True
unecessary_stop_words contains she False
### shape ###
nr rows: 18240 | nr cols: 16


In [ ]:
# Check tokenized tweets

for row in df.no_stopwords[0:10]:  # iterating through the rows of the object column
    print(row, '\n')

['@theblaze', '@realdonaldtrump'] 

['@barackobama', '@fbi', '@lorettalynch', 'collusion', 'together', '#nojustice', '@realdonaldtrump', '#trumppence'] 

['@theblaze', '@realdonaldtrump'] 

['#cnn', '#newday', 'clear', '#trump', 'deliberately', 'throwing', 'race', ',', '2007', 'he', 'knew', '#isis', 'destabilization', 'mideast', 'started', 'w', '/', 'iraq', 'invasion'] 

['@realdonaldtrump', ',', 'recognize', 'lie', 'came', 'mouth', ',', 'continually', '.', '#nevertrump'] 

['"', 'kid', ',', 'know', ',', 'suing', 'someone', '?', 'thats', 'beautiful', 'thing', '1', 'human', 'could', 'another', 'human', '"', '@funnyordie', '@realdonaldtrump', '😂', '💩'] 

['@hillaryclinton', 'co-founder', 'isis', ',', 'crooked', ',', 'evil', ',', 'lying', ',', 'witch', '.', 'live', '?'] 

['@mike4193496', '@realdonaldtrump', 'totally', 'concur', '!', '!', 'election', 'cra', 'cra', 'n', 'corruption', 'gov', 'mind', 'blowing', '!', '!', 'trump', '=', 'last', 'hope', '!', '!', '!'] 

['@realdonaldtrump', '@e

In [ ]:
punctuations = list(string.punctuation) 

# Remove punctuation
df['no_punctuation'] = df['no_stopwords'].apply(lambda tweet: [word for word in tweet if word not in punctuations]) 

# Remove numbers, but not words that contain numbers.
df['no_digits'] = df['no_punctuation'].apply(lambda tweet: [word for word in tweet if not word.isnumeric()]) 

# Remove words that are only one character.
df['before_stemmitizing'] = df['no_digits'].apply(lambda tweet: [word for word in tweet if len(word) > 1]) 

print("### shape ###")
print("nr rows:", df.shape[0], "| nr cols:", df.shape[1])

### shape ###
nr rows: 18240 | nr cols: 19


In [ ]:
# Check tokenized tweets

for row in df.before_stemmitizing[0:10]:  # iterating through the rows of the object column
    print(row, '\n')

['@theblaze', '@realdonaldtrump'] 

['@barackobama', '@fbi', '@lorettalynch', 'collusion', 'together', '#nojustice', '@realdonaldtrump', '#trumppence'] 

['@theblaze', '@realdonaldtrump'] 

['#cnn', '#newday', 'clear', '#trump', 'deliberately', 'throwing', 'race', 'he', 'knew', '#isis', 'destabilization', 'mideast', 'started', 'iraq', 'invasion'] 

['@realdonaldtrump', 'recognize', 'lie', 'came', 'mouth', 'continually', '#nevertrump'] 

['kid', 'know', 'suing', 'someone', 'thats', 'beautiful', 'thing', 'human', 'could', 'another', 'human', '@funnyordie', '@realdonaldtrump'] 

['@hillaryclinton', 'co-founder', 'isis', 'crooked', 'evil', 'lying', 'witch', 'live'] 

['@mike4193496', '@realdonaldtrump', 'totally', 'concur', 'election', 'cra', 'cra', 'corruption', 'gov', 'mind', 'blowing', 'trump', 'last', 'hope'] 

['@realdonaldtrump', '@elsolarverde', 'issues', 'idiot', 'claim', 'she', 'founded', 'isis', 'trump', 'go', 'hell', 'lying', 'stealing', 'shame'] 

["can't", 'stand', '@hillarycl

### 4. Stemming tokens

In [ ]:
def stemmitize(token):
    ps = PorterStemmer() #stemmization 
    stemmed_words = [ps.stem(w) for w in token] #stemmization 
    return " ".join(stemmed_words)

df['final'] = df['before_stemmitizing'].apply(stemmitize)

In [ ]:
print("nr rows:", df.shape[0], "| nr cols:", df.shape[1])

for row in df.final[0:10]:  # iterating through the rows of the object column
    print(row, '\n')

nr rows: 18240 | nr cols: 20
@theblaz @realdonaldtrump 

@barackobama @fbi @lorettalynch collus togeth #nojustic @realdonaldtrump #trumppenc 

@theblaz @realdonaldtrump 

#cnn #newday clear #trump deliber throw race he knew #isi destabil mideast start iraq invas 

@realdonaldtrump recogn lie came mouth continu #nevertrump 

kid know su someon that beauti thing human could anoth human @funnyordi @realdonaldtrump 

@hillaryclinton co-found isi crook evil lie witch live 

@mike4193496 @realdonaldtrump total concur elect cra cra corrupt gov mind blow trump last hope 

@realdonaldtrump @elsolarverd issu idiot claim she found isi trump go hell lie steal shame 

can't stand @hillaryclinton @realdonaldtrump take look win ... settl #15for15 #teamgov #youin 



### Save dataset

In [ ]:
# Remove processing columns
df = df.drop(columns=['country', 'text_has_tag', 'text_no_http', 'tokens', 'no_stopwords', 'no_punctuation', 'no_digits'])

print("### describe ###")
print(df.describe(include='all'))
print("### df.isnull().sum() ###")
print(df.isnull().sum())

### describe ###
                    text in_reply_to_screen_name  followers_count  \
count              18240                   12139     1.824000e+04   
unique             18100                    3108              NaN   
top     @realDonaldTrump         realDonaldTrump              NaN   
freq                  20                    4016              NaN   
mean                 NaN                     NaN     2.736047e+03   
std                  NaN                     NaN     8.102227e+04   
min                  NaN                     NaN     0.000000e+00   
25%                  NaN                     NaN     7.300000e+01   
50%                  NaN                     NaN     3.470000e+02   
75%                  NaN                     NaN     1.400250e+03   
max                  NaN                     NaN     1.085954e+07   

        retweet_count                      created_at        id_str  \
count         18240.0                           18240  1.824000e+04   
unique      

In [ ]:
# Remove test data
filename = 'test_data.csv'
test_df = pd.read_csv(filename, encoding ='utf_8')
print("### test data shape ###")
print("nr rows:", test_df.shape[0], "| nr cols:", test_df.shape[1])

test_texts = test_df['text'].unique()

def is_test_data(text: str) -> bool:
    return text in test_texts

print("### data shape before ###")
print("nr rows:", df.shape[0], "| nr cols:", df.shape[1])
df['is_test_data'] = df['text'].apply(is_test_data)
df = df[df.is_test_data != True]
print("### data shape after ###")
print("nr rows:", df.shape[0], "| nr cols:", df.shape[1])
print("### describe ###")
print(df.describe(include='all'))
print("### df.isnull().sum() ###")
print(df.isnull().sum())
df = df.drop(columns=['is_test_data'])

### test data shape ###
nr rows: 286 | nr cols: 17
### data shape before ###
nr rows: 18240 | nr cols: 13
### data shape after ###
nr rows: 18239 | nr cols: 14
### describe ###
                    text in_reply_to_screen_name  followers_count  \
count              18239                   12138     1.823900e+04   
unique             18099                    3108              NaN   
top     @realDonaldTrump         realDonaldTrump              NaN   
freq                  20                    4015              NaN   
mean                 NaN                     NaN     2.736083e+03   
std                  NaN                     NaN     8.102450e+04   
min                  NaN                     NaN     0.000000e+00   
25%                  NaN                     NaN     7.300000e+01   
50%                  NaN                     NaN     3.470000e+02   
75%                  NaN                     NaN     1.400000e+03   
max                  NaN                     NaN     1.085954e+0

In [ ]:
# Write csv file
output_file_name=r'C:\Users\Marta\Dropbox\2019-Move-to-NL\UvA\fundamentals-data-science\assignments\assignment1\processed_data.csv'
df.to_csv(output_file_name', index = False)